In [1]:
import h5py
import numpy as np

In [10]:
def hdf2dict(file_name):
    # Define the dictionary to store the datasets
    data = {}
    #file_name = 'macro421_UO2_03__900K.h5'
    with h5py.File("..//02.Macro.XS.421g/" + file_name, "r") as file:
        # Iterate over the dataset names in the group
        for dataset_name in file.keys():
            # Read the dataset
            dataset = file[dataset_name]
            # Check if the dataset is a struct
            if isinstance(dataset, h5py.Group):
                # Create a dictionary to store the struct fields
                struct_data = {}
                # Iterate over the fields in the struct
                for field_name in dataset.keys():
                    # Read the field dataset
                    field_dataset = np.array(dataset[field_name])
                    # Store the field dataset in the struct dictionary
                    struct_data[field_name] = field_dataset
                # Store the struct data in the main data dictionary
                data[dataset_name] = struct_data
            else:
                # Read the dataset as a regular array
                dataset_array = np.array(dataset)
                # Store the dataset array in the dictionary
                data[dataset_name] = dataset_array

#print(data["SigC"])
    return data

#file_name = 'macro421_UO2_03__900K.h5'
#fuel = hdf2dict(file_name)
#fuel.keys()

dict_keys(['SigC', 'SigF', 'SigL', 'SigP', 'SigT', 'aw', 'chi', 'den', 'eg', 'fissile', 'isoName', 'ng', 'numDen', 'sig0', 'sig2_G', 'sigS_G', 'temp'])

In [12]:
# Start stopwatch
tic = 0  # Placeholder for stopwatch functionality in Python

#--------------------------------------------------------------------------
# Number of source neutrons
numNeutrons_born = 10      # INPUT

# Number of inactive source cycles to skip before starting k-eff accumulation
numCycles_inactive = 10    # INPUT

# Number of active source cycles for k-eff accumulation
numCycles_active = 10     # INPUT

# Size of the square unit cell
pitch = 3.6  # cm           # INPUT

#--------------------------------------------------------------------------
# Path to macroscopic cross section data:
# (Assuming the corresponding data files are available and accessible)
macro_xs_path = '..//02.Macro.XS.421g'

# Fill the structures fuel, clad, and cool with the cross-section data
fuel = hdf2dict('macro421_UO2_03__900K.h5')  # INPUT
print(f"File 'macro421_UO2_03__900K.h5' has been read in.")
clad = hdf2dict('macro421_Zry__600K.h5')     # INPUT
print(f"File 'macro421_Zry__600K.h5' has been read in.")
cool  = hdf2dict('macro421_H2OB__600K.h5')   # INPUT
print(f"File 'macro421_H2OB__600K.h5' has been read in.")

# Define the majorant: the maximum total cross-section vector
SigTmax = np.max([fuel['SigT'], clad['SigT'], cool['SigT']])

# Number of energy groups
ng = fuel['ng']

#--------------------------------------------------------------------------
# Detectors
detectS = np.zeros(ng)

#--------------------------------------------------------------------------
# Four main vectors describing the neutrons in a batch
x = np.zeros(numNeutrons_born * 2)
y = np.zeros(numNeutrons_born * 2)
weight = np.ones(numNeutrons_born * 2)
iGroup = np.ones(numNeutrons_born * 2, dtype=int)

#--------------------------------------------------------------------------
# Neutrons are assumed born randomly distributed in the cell with weight 1
# with sampled fission energy spectrum
numNeutrons = numNeutrons_born
for iNeutron in range(numNeutrons):
    x[iNeutron] = np.random.rand() * pitch
    y[iNeutron] = np.random.rand() * pitch
    weight[iNeutron] = 1
    # Sample the neutron energy group
    iGroup[iNeutron] = np.argmax(np.cumsum(fuel['chi']) >= np.random.rand()) + 1

#--------------------------------------------------------------------------
# Prepare vectors for keff and standard deviation of keff
keff_expected = np.ones(numCycles_active)
sigma_keff = np.zeros(numCycles_active)
keff_active_cycle = np.ones(numCycles_active)
virtualCollision = False

# Main (power) iteration loop
for iCycle in range(1, numCycles_inactive + numCycles_active + 1):

    # Normalize the weights of the neutrons to make the total weight equal to
    # numNeutrons_born (equivalent to division by keff_cycle)
    weight = (weight / np.sum(weight, axis=0, keepdims=True)) * numNeutrons_born
    weight0 = weight.copy()

    #----------------------------------------------------------------------
    # Loop over neutrons
    for iNeutron in range(numNeutrons):

        absorbed = False

        #------------------------------------------------------------------
        # Neutron random walk cycle: from emission to absorption

        while not absorbed:

            # Sample free path length according to the Woodcock method
            freePath = -np.log(np.random.rand()) / SigTmax[iGroup[iNeutron]]

            if not virtualCollision:
                # Sample the direction of neutron flight assuming both
                # fission and scattering are isotropic in the lab (a strong
                # assumption!)
                teta = np.pi * np.random.rand()
                phi = 2.0 * np.pi * np.random.rand()
                dirX = np.sin(teta) * np.cos(phi)
                dirY = np.sin(teta) * np.sin(phi)

            # Fly
            x[iNeutron] += freePath * dirX
            y[iNeutron] += freePath * dirY

            # If outside the cell, find the corresponding point inside the
            # cell
            while x[iNeutron] < 0:
                x[iNeutron] += pitch
            while y[iNeutron] < 0:
                y[iNeutron] += pitch
            while x[iNeutron] > pitch:
                x[iNeutron] -= pitch
            while y[iNeutron] > pitch:
                y[iNeutron] -= pitch

            # Find the total and scattering cross sections
            if 0.9 < x[iNeutron] < 2.7:  # INPUT
                SigA = fuel['SigF'][iGroup[iNeutron]] + fuel['SigC'][iGroup[iNeutron]] + fuel['SigL'][iGroup[iNeutron]]
                SigS = fuel['SigS'][0][iGroup[iNeutron], :].reshape(-1, 1)
                SigP = fuel['SigP'][iGroup[iNeutron]]
            elif x[iNeutron] < 0.7 or x[iNeutron] > 2.9:  # INPUT
                SigA = cool['SigC'][iGroup[iNeutron]] + cool['SigL'][iGroup[iNeutron]]
                SigS = cool['SigS'][0][iGroup[iNeutron], :].reshape(-1, 1)
                SigP = 0
            else:
                SigA = clad['SigC'][iGroup[iNeutron]] + clad['SigL'][iGroup[iNeutron]]
                SigS = clad['SigS'][0][iGroup[iNeutron], :].reshape(-1, 1)
                SigP = 0

            # Find the other cross sections ...
            # ... scattering
            SigS_sum = np.sum(SigS)
            # ... total
            SigT = SigA + SigS_sum
            # ... virtual
            SigV = SigTmax[iGroup[iNeutron]] - SigT

            # Sample the type of the collision: virtual (do nothing) or real
            if SigV / SigTmax[iGroup[iNeutron]] >= np.random.rand():  # virtual collision

                virtualCollision = True

            else:  # real collision

                virtualCollision = False

                # Sample type of the collision: scattering or absorption
                if SigS_sum / SigT >= np.random.rand():  # isotropic scattering

                    # Score scatterings with account for weight divided by the
                    # total scattering cross section
                    detectS[iGroup[iNeutron]] += weight[iNeutron] / SigS_sum

                    # Sample the energy group of the secondary neutron
                    iGroup[iNeutron] = np.argmax(np.cumsum(SigS) / SigS_sum >= np.random.rand()) + 1

                else:  # absorption

                    absorbed = True

                    # Neutron is converted to the new fission neutron with
                    # the weight increased by eta
                    weight[iNeutron] *= SigP / SigA

                    # Sample the energy group for the new-born neutron
                    iGroup[iNeutron] = np.argmax(np.cumsum(fuel['chi']) >= np.random.rand()) + 1

        # End of neutron random walk cycle: from emission to absorption
    # End of loop over neutrons


fuel.close()
clad.close()
cool.close()


File 'macro421_UO2_03__900K.h5' has been read in.
File 'macro421_Zry__600K.h5' has been read in.
File 'macro421_H2OB__600K.h5' has been read in.


AxisError: axis 1 is out of bounds for array of dimension 1

In [14]:
SigTmax[]

39.6051260523968